# 📊 Exploration des Données - Projet Trustworthy AI

**Objectif**: Explorer et analyser les données d'images faciales des 3 membres du groupe pour préparer l'apprentissage fédéré.

## 🎯 Contenu du notebook
1. Chargement et configuration
2. Exploration des dossiers de données
3. Visualisation des images
4. Analyse de la qualité des données
5. Statistiques du dataset
6. Préparation pour l'apprentissage fédéré

## 1. 📚 Imports et Configuration

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from collections import Counter
import warnings

# Configuration
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set_style("whitegrid")
warnings.filterwarnings('ignore')

# Chemin des données
DATA_PATH = "../data/raw"

print("🔧 Configuration terminée!")

🔧 Configuration terminée!


## 2. 🔍 Exploration de la Structure des Données

In [ ]:
def explore_data_structure(data_path):
    """Explore la structure des dossiers de données"""
    
    print("📁 Structure du dossier de données:")
    print(f"Chemin principal: {data_path}")
    print("-" * 50)
    
    if not os.path.exists(data_path):
        print("❌ Le dossier de données n'existe pas!")
        print("💡 Créez les dossiers et ajoutez vos photos:")
        print("   - data/raw/member1/")
        print("   - data/raw/member2/")
        print("   - data/raw/member3/")
        return None
    
    data_info = {}
    
    for member_folder in sorted(os.listdir(data_path)):
        member_path = os.path.join(data_path, member_folder)
        
        if os.path.isdir(member_path) and member_folder.startswith('member'):
            # Compter les images
            image_files = [f for f in os.listdir(member_path) 
                          if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            
            data_info[member_folder] = {
                'path': member_path,
                'num_images': len(image_files),
                'image_files': image_files
            }
            
            print(f"👤 {member_folder}: {len(image_files)} images")
    
    total_images = sum([info['num_images'] for info in data_info.values()])
    print(f"\n📊 Total: {total_images} images pour {len(data_info)} membres")
    
    return data_info

# Explorer la structure
data_info = explore_data_structure(DATA_PATH)

## 3. 🖼️ Visualisation des Échantillons d'Images

In [ ]:
def visualize_sample_images(data_info, samples_per_member=3):
    """Affiche des échantillons d'images pour chaque membre"""
    
    if data_info is None:
        print("❌ Impossible d'afficher les images - données manquantes")
        return
    
    num_members = len(data_info)
    
    fig, axes = plt.subplots(num_members, samples_per_member, 
                            figsize=(15, 5 * num_members))
    
    if num_members == 1:
        axes = axes.reshape(1, -1)
    
    for i, (member_name, member_data) in enumerate(data_info.items()):
        image_files = member_data['image_files']
        
        # Sélectionner des échantillons aléatoires
        sample_files = np.random.choice(image_files, 
                                      min(samples_per_member, len(image_files)), 
                                      replace=False)
        
        for j, img_file in enumerate(sample_files):
            img_path = os.path.join(member_data['path'], img_file)
            
            try:
                # Charger et afficher l'image
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                
                axes[i, j].imshow(image)
                axes[i, j].set_title(f"{member_name}\n{img_file}", fontsize=10)
                axes[i, j].axis('off')
                
            except Exception as e:
                axes[i, j].text(0.5, 0.5, f"Erreur:\n{img_file}", 
                               ha='center', va='center', transform=axes[i, j].transAxes)
                axes[i, j].axis('off')
        
        # Masquer les axes supplémentaires s'il y a moins d'images
        for j in range(len(sample_files), samples_per_member):
            axes[i, j].axis('off')
    
    plt.suptitle("📸 Échantillons d'Images par Membre", fontsize=16, y=0.98)
    plt.tight_layout()
    plt.show()

# Visualiser les échantillons
visualize_sample_images(data_info, samples_per_member=4)

## 4. 📏 Analyse des Dimensions et Qualité des Images

In [ ]:
def analyze_image_properties(data_info):
    """Analyse les propriétés des images (dimensions, format, qualité)"""
    
    if data_info is None:
        print("❌ Impossible d'analyser - données manquantes")
        return None
    
    analysis_results = []
    
    print("🔍 Analyse des propriétés des images...")
    
    for member_name, member_data in data_info.items():
        print(f"\n👤 Analyse de {member_name}:")
        
        for img_file in member_data['image_files']:
            img_path = os.path.join(member_data['path'], img_file)
            
            try:
                # Ouvrir avec PIL pour les métadonnées
                with Image.open(img_path) as img:
                    width, height = img.size
                    format_img = img.format
                    mode = img.mode
                
                # Taille du fichier
                file_size = os.path.getsize(img_path) / 1024  # KB
                
                analysis_results.append({
                    'member': member_name,
                    'filename': img_file,
                    'width': width,
                    'height': height,
                    'aspect_ratio': width / height,
                    'format': format_img,
                    'mode': mode,
                    'file_size_kb': file_size,
                    'resolution': width * height
                })
                
            except Exception as e:
                print(f"  ❌ Erreur avec {img_file}: {e}")
    
    if analysis_results:
        df = pd.DataFrame(analysis_results)
        return df
    else:
        return None

# Analyser les propriétés
df_analysis = analyze_image_properties(data_info)

if df_analysis is not None:
    print("\n📊 Résumé statistique:")
    print(df_analysis.groupby('member').agg({
        'width': ['mean', 'std', 'min', 'max'],
        'height': ['mean', 'std', 'min', 'max'],
        'file_size_kb': ['mean', 'std', 'min', 'max'],
        'aspect_ratio': ['mean', 'std']
    }).round(2))

## 5. 📊 Visualisations Statistiques

In [ ]:
def create_statistical_visualizations(df_analysis, data_info):
    """Crée des visualisations statistiques du dataset"""
    
    if df_analysis is None or data_info is None:
        print("❌ Données insuffisantes pour les visualisations")
        return
    
    # Configuration des couleurs
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
    
    # 1. Distribution du nombre d'images par membre
    plt.figure(figsize=(15, 12))
    
    # Subplot 1: Nombre d'images par membre
    plt.subplot(2, 3, 1)
    member_counts = df_analysis['member'].value_counts()
    bars = plt.bar(member_counts.index, member_counts.values, color=colors)
    plt.title('📊 Nombre d\'Images par Membre')
    plt.ylabel('Nombre d\'images')
    plt.xticks(rotation=45)
    
    # Ajouter les valeurs sur les barres
    for bar, value in zip(bars, member_counts.values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                str(value), ha='center', va='bottom')
    
    # Subplot 2: Distribution des largeurs
    plt.subplot(2, 3, 2)
    for i, member in enumerate(df_analysis['member'].unique()):
        member_data = df_analysis[df_analysis['member'] == member]
        plt.hist(member_data['width'], alpha=0.7, label=member, 
                color=colors[i], bins=10)
    plt.title('📏 Distribution des Largeurs')
    plt.xlabel('Largeur (pixels)')
    plt.ylabel('Fréquence')
    plt.legend()
    
    # Subplot 3: Distribution des hauteurs
    plt.subplot(2, 3, 3)
    for i, member in enumerate(df_analysis['member'].unique()):
        member_data = df_analysis[df_analysis['member'] == member]
        plt.hist(member_data['height'], alpha=0.7, label=member, 
                color=colors[i], bins=10)
    plt.title('📐 Distribution des Hauteurs')
    plt.xlabel('Hauteur (pixels)')
    plt.ylabel('Fréquence')
    plt.legend()
    
    # Subplot 4: Ratio d'aspect
    plt.subplot(2, 3, 4)
    sns.boxplot(data=df_analysis, x='member', y='aspect_ratio', palette=colors)
    plt.title('📱 Ratio d\'Aspect par Membre')
    plt.ylabel('Ratio (largeur/hauteur)')
    plt.xticks(rotation=45)
    
    # Subplot 5: Taille des fichiers
    plt.subplot(2, 3, 5)
    sns.boxplot(data=df_analysis, x='member', y='file_size_kb', palette=colors)
    plt.title('💾 Taille des Fichiers par Membre')
    plt.ylabel('Taille (KB)')
    plt.xticks(rotation=45)
    
    # Subplot 6: Résolution totale
    plt.subplot(2, 3, 6)
    sns.scatterplot(data=df_analysis, x='width', y='height', 
                   hue='member', palette=colors, s=50)
    plt.title('🎯 Largeur vs Hauteur')
    plt.xlabel('Largeur (pixels)')
    plt.ylabel('Hauteur (pixels)')
    
    plt.tight_layout()
    plt.show()
    
    # Tableau de résumé
    print("\n📋 Résumé du Dataset:")
    summary_table = df_analysis.groupby('member').agg({
        'filename': 'count',
        'width': 'mean',
        'height': 'mean',
        'file_size_kb': 'mean',
        'aspect_ratio': 'mean'
    }).round(1)
    
    summary_table.columns = ['Nb Images', 'Largeur Moy.', 'Hauteur Moy.', 'Taille Moy. (KB)', 'Ratio Moy.']
    print(summary_table)

# Créer les visualisations
create_statistical_visualizations(df_analysis, data_info)

## 6. 🔒 Analyse pour l'Apprentissage Fédéré

In [ ]:
def analyze_for_federated_learning(data_info, df_analysis):
    """Analyse la répartition des données pour l'apprentissage fédéré"""
    
    print("🔒 Analyse pour l'Apprentissage Fédéré")
    print("=" * 50)
    
    if data_info is None:
        print("❌ Données manquantes")
        return
    
    total_images = sum([info['num_images'] for info in data_info.values()])
    num_clients = len(data_info)
    
    print(f"📊 Configuration Fédérée:")
    print(f"   • Nombre de clients: {num_clients}")
    print(f"   • Total d'images: {total_images}")
    print(f"   • Moyenne par client: {total_images/num_clients:.1f} images")
    
    # Distribution des données
    print(f"\n👥 Répartition par client:")
    for i, (member_name, member_data) in enumerate(data_info.items(), 1):
        num_images = member_data['num_images']
        percentage = (num_images / total_images) * 100
        print(f"   Client {i} ({member_name}): {num_images} images ({percentage:.1f}%)")
    
    # Recommandations pour l'apprentissage fédéré
    print(f"\n💡 Recommandations:")
    
    # Vérifier l'équilibre des données
    image_counts = [info['num_images'] for info in data_info.values()]
    std_dev = np.std(image_counts)
    mean_count = np.mean(image_counts)
    cv = std_dev / mean_count  # Coefficient de variation
    
    if cv < 0.2:
        print("   ✅ Distribution équilibrée des données")
    elif cv < 0.5:
        print("   ⚠️ Distribution moyennement équilibrée")
        print("   💡 Considérez l'augmentation des données pour les clients avec moins d'images")
    else:
        print("   ❌ Distribution déséquilibrée")
        print("   💡 Recommandation forte: augmentation de données ou rééquilibrage")
    
    # Minimum recommandé
    min_images = min(image_counts)
    if min_images < 10:
        print(f"   ⚠️ Client avec seulement {min_images} images - minimum recommandé: 20")
    elif min_images < 20:
        print(f"   ⚠️ Minimum d'images acceptable mais bas ({min_images})")
    else:
        print(f"   ✅ Nombre d'images suffisant (min: {min_images})")
    
    # Stratégie d'entraînement
    print(f"\n🎯 Stratégies recommandées:")
    print(f"   • Taille de batch: 8-16 (données limitées)")
    print(f"   • Augmentation de données: Fortement recommandée")
    print(f"   • Rounds fédérés: 50-100")
    print(f"   • Epochs locaux: 3-5 par round")
    
    # Confidentialité
    print(f"\n🔒 Aspects de confidentialité:")
    print(f"   • Données sensibles: Images faciales identifiables")
    print(f"   • Differential Privacy: Recommandée (ε=1.0)")
    print(f"   • Secure Aggregation: Obligatoire")
    print(f"   • Chiffrement des communications: TLS 1.3")

# Analyser pour l'apprentissage fédéré
analyze_for_federated_learning(data_info, df_analysis)

## 7. ✅ Checklist de Préparation

In [ ]:
def create_preparation_checklist(data_info, df_analysis):
    """Crée une checklist pour la préparation des données"""
    
    print("✅ CHECKLIST DE PRÉPARATION")
    print("=" * 50)
    
    checklist_items = []
    
    # 1. Présence des données
    if data_info is not None:
        checklist_items.append(("✅", "Dossiers de données créés"))
        
        # Vérifier chaque membre
        for i in range(1, 4):
            member_key = f'member{i}'
            if member_key in data_info:
                num_images = data_info[member_key]['num_images']
                if num_images > 0:
                    checklist_items.append(("✅", f"Images pour {member_key}: {num_images} fichiers"))
                else:
                    checklist_items.append(("❌", f"Aucune image pour {member_key}"))
            else:
                checklist_items.append(("❌", f"Dossier {member_key} manquant"))
    else:
        checklist_items.append(("❌", "Dossiers de données manquants"))
    
    # 2. Qualité des données
    if df_analysis is not None:
        # Vérifier les formats
        formats = df_analysis['format'].unique()
        supported_formats = {'JPEG', 'PNG', 'JPG'}
        if all(fmt in supported_formats for fmt in formats):
            checklist_items.append(("✅", f"Formats d'images supportés: {', '.join(formats)}"))
        else:
            checklist_items.append(("⚠️", f"Formats mixtes détectés: {', '.join(formats)}"))
        
        # Vérifier les tailles
        min_size = df_analysis[['width', 'height']].min().min()
        if min_size >= 224:
            checklist_items.append(("✅", f"Résolution suffisante (min: {min_size}px)"))
        else:
            checklist_items.append(("⚠️", f"Résolution faible détectée (min: {min_size}px)"))
    else:
        checklist_items.append(("❌", "Analyse de qualité non effectuée"))
    
    # 3. Étapes suivantes
    next_steps = [
        ("📝", "Preprocessing des images (redimensionnement, normalisation)"),
        ("🔄", "Augmentation des données si nécessaire"),
        ("🔀", "Division des données pour l'apprentissage fédéré"),
        ("🧠", "Construction et test du modèle CNN"),
        ("🔗", "Implémentation de l'apprentissage fédéré"),
        ("🔍", "Application des techniques d'explicabilité"),
        ("🔒", "Test des mesures de confidentialité")
    ]
    
    # Affichage
    print("\n📊 État actuel:")
    for status, item in checklist_items:
        print(f"   {status} {item}")
    
    print("\n🚀 Prochaines étapes:")
    for status, step in next_steps:
        print(f"   {status} {step}")
    
    # Recommandations finales
    print("\n💡 Recommandations avant de continuer:")
    if data_info is None:
        print("   1. 📸 Collectez au moins 20-30 photos par membre")
        print("   2. 📁 Organisez les photos dans data/raw/member1/, member2/, member3/")
        print("   3. 🖼️ Assurez-vous que les photos sont de bonne qualité")
    else:
        total_images = sum([info['num_images'] for info in data_info.values()])
        if total_images < 60:
            print("   1. 📸 Collectez plus de photos (recommandé: 20+ par membre)")
        print("   2. 🔍 Vérifiez la qualité et la diversité des photos")
        print("   3. ▶️ Procédez au preprocessing (notebook 02)")

# Créer la checklist
create_preparation_checklist(data_info, df_analysis)

## 🎯 Conclusion

Cette exploration nous a permis de:

1. **📁 Structure des données**: Vérification de l'organisation des dossiers
2. **📊 Statistiques**: Analyse de la distribution et qualité des images
3. **🔍 Qualité**: Évaluation des dimensions, formats et tailles
4. **🔒 Préparation FL**: Analyse pour l'apprentissage fédéré
5. **✅ Checklist**: Validation de l'état de préparation

### 🚀 Prochaines étapes

1. **Notebook 02**: Preprocessing et augmentation des données
2. **Notebook 03**: Entraînement du modèle CNN
3. **Notebook 04**: Implémentation de l'apprentissage fédéré
4. **Notebook 05**: Techniques d'explicabilité (LIME, SHAP, Grad-CAM)
5. **Notebook 06**: Évaluation de la confidentialité et sécurité

---

**📝 Note**: Ce notebook fait partie du projet Trustworthy AI pour la classification faciale éthique avec apprentissage fédéré.